In [ ]:
import numpy as np
import pickle
import healpy as hp
import matplotlib.pyplot as plt
import os
import subprocess
import sys
import scipy
from scipy.optimize import minimize
from scipy import stats
from matplotlib.ticker import FormatStrFormatter
from getdist import plots, MCSamples
import getdist

In [ ]:
font = {'size'   : 20, 'family':'STIXGeneral'}
plt.rcParams.update({
     'text.usetex': True,
     'font.family': 'serif',
     'font.sans-serif': ['Computer Modern'],
     'font.size':20})
plt.rc_context({'axes.autolimit_mode': 'round_numbers'})

getdist.plots.GetDistPlotSettings.legend_fontsize = 14
# plt.style.use('dark_background')

In [ ]:
save = False #whether to save plots
Nsims_lfi = 30000 #number of sims used in LFI
tsz_type = 'nongaussian' #either 'gaussian' or 'nongaussian' (whichever was used in the yaml files)
Nsims = 2000 #number of sims used in Gaussian likelihood
names = [r'$A_{\mathrm{CMB}}$', r'$A_{\mathrm{ftSZ}}$']
labels = names

In [ ]:
def ellipse_area(sample):
    '''
    ARGUMENTS
    ---------
    sample: MCSamples object containing posteriors

    RETURNS
    ---------
    area: float, area of 68% confidence interval Fisher ellipse
    
    '''
    cov = sample.cov()
    sigmax = np.sqrt(cov[0,0])
    sigmay = np.sqrt(cov[1,1])
    sigmaxy = cov[0,1]
    a = np.sqrt((sigmax**2+sigmay**2)/2 + np.sqrt(((sigmax**2-sigmay**2)**2)/4+sigmaxy**2))
    b = np.sqrt((sigmax**2+sigmay**2)/2 - np.sqrt(((sigmax**2-sigmay**2)**2)/4+sigmaxy**2))
    area = np.pi*a*b*2.3 # fixed from 1.52
    return area

def tsz_spectral_response(freqs):
    '''
    ARGUMENTS
    ---------
    freqs: 1D numpy array, contains frequencies (GHz) for which to calculate tSZ spectral response

    RETURNS
    ---------
    1D array containing tSZ spectral response to each frequency (units of K_CMB)
    '''
    T_cmb = 2.726
    T_cmb_uK = 2.726e6
    h = 6.62607004*10**(-34)
    kb = 1.38064852*10**(-23)
    response = []
    for freq in freqs:
        x = h*(freq*10**9)/(kb*T_cmb) #x is v/56.9 GHz
        response.append(T_cmb*(x*1/np.tanh(x/2)-4))
    return np.array(response)

# Gaussian Likelihood

### Multifrequency

In [ ]:
freq_strs = ['90_150', '90_120_150', '90_280', '280_353', '280_353_400' , '90_150_280', '90_150_220']

samples = []
for freq_str in freq_strs:
    samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_multifrequency_{tsz_type}tsz_{Nsims//1000}ksims_tszamp150_gaussianlkl.p', 'rb')).T, names = names, labels = labels, label=f'Mult. PS Gauss. Lkl.\n{freq_str.replace("_", ", ")}'))

g = plots.get_subplot_plotter()
g.triangle_plot(samples, filled=True)
plt.savefig('mult_gausslkl_freq_comparison.pdf')


### HILC (weights once)

In [ ]:
freq_strs = ['90_150', '90_120_150', '90_280', '280_353', '280_353_400' , '90_150_280', '90_150_220']

samples = []
for freq_str in freq_strs:
    samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_HILC_{tsz_type}tsz_weightsonce_{Nsims//1000}ksims_tszamp150_gaussianlkl_analytic.p', 'rb')).T, names = names, labels = labels, label=f'HILC (weights once)\n Gauss. Lkl. {freq_str.replace("_", ", ")}'))

    
g = plots.get_subplot_plotter()

g.triangle_plot(samples, filled=True)
plt.savefig('hilc_wtsonce_gausslkl_freq_comparison.pdf')

### HILC (weights vary)

In [ ]:
freq_strs = ['90_150', '90_120_150', '90_280', '280_353', '280_353_400' , '90_150_280', '90_150_220']

samples = []
for freq_str in freq_strs:
    samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_HILC_{tsz_type}tsz_weightsvary_{Nsims//1000}ksims_tszamp150_gaussianlkl_SR.p', 'rb')).T, names = names, labels = labels, label=f'HILC (weights vary)\n Gauss. Lkl. {freq_str.replace("_", ", ")}'))

g = plots.get_subplot_plotter()

g.triangle_plot(samples, filled=True)
plt.savefig('hilc_wtsvary_gausslkl_freq_comparison.pdf')

### NILC

In [ ]:
freq_strs = ['90_150', '90_120_150', '90_280', '280_353', '280_353_400' , '90_150_280', '90_150_220']


samples = []
for freq_str in freq_strs:
    samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_NILC_{tsz_type}tsz_{Nsims//1000}ksims_tszamp150_gaussianlkl_4scales.p', 'rb')).T, names = names, labels = labels, label=f'NILC gausslkl {freq_str}'))
    
g = plots.get_subplot_plotter()

g.triangle_plot(samples, filled=True)
plt.savefig('nilc_gausslkl_freq_comparison.pdf')



### Separate plot for each frequency combination

In [ ]:
freq_strs = ['90_120', '90_150', '90_280', '90_120_150', '90_150_280', '90_150_220', '280_353', '280_353_400']

for i, freq_str in enumerate(freq_strs):
    samples = []
    samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_multifrequency_{tsz_type}tsz_{Nsims//1000}ksims_tszamp150_gaussianlkl.p', 'rb')).T, names = names, labels = labels, label=f'Mult. PS {freq_str.replace("_", ", ")}'))
    samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_HILC_{tsz_type}tsz_weightsonce_{Nsims//1000}ksims_tszamp150_gaussianlkl_analytic.p', 'rb')).T, names = names, labels = labels, label=f'HILC (weights once) {freq_str.replace("_", ", ")}'))
    samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_HILC_{tsz_type}tsz_weightsvary_{Nsims//1000}ksims_tszamp150_gaussianlkl_SR.p', 'rb')).T, names = names, labels = labels, label=f'HILC (weights vary) {freq_str.replace("_", ", ")}'))
    samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_NILC_{tsz_type}tsz_{Nsims//1000}ksims_tszamp150_gaussianlkl_4scales.p', 'rb')).T, names = names, labels = labels, label=f'NILC {freq_str.replace("_", ", ")}'))

    g = plots.get_subplot_plotter()
    g.triangle_plot(samples, filled=True)
    plt.savefig(f'{freq_strs[i]}_gausslkl.pdf')

## Ellipse Areas

### Multifrequency Ellipse Areas

In [ ]:
freq_strs = ['90_120', '90_150', '90_280', '90_120_150', '90_150_280', '90_150_220', '280_353', '280_353_400']

samples = []
for i, freq_str in enumerate(freq_strs):
    samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_multifrequency_{tsz_type}tsz_{Nsims//1000}ksims_tszamp150_gaussianlkl.p', 'rb')).T, names = names, labels = labels, label=f'Mult. PS {freq_str.replace("_", ", ")}'))

for s,sample in enumerate(samples):
    print(f'1000*area for {freq_strs[s]}: {1000*ellipse_area(sample)}')


### HILC (weights once) Ellipse Areas

In [ ]:
freq_strs = ['90_120', '90_150', '90_280', '90_120_150', '90_150_280', '90_150_220', '280_353', '280_353_400']

samples = []
for i, freq_str in enumerate(freq_strs):
    samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_HILC_{tsz_type}tsz_weightsonce_{Nsims//1000}ksims_tszamp150_gaussianlkl_analytic.p', 'rb')).T, names = names, labels = labels, label=f'HILC (weights once) {freq_str.replace("_", ", ")}'))

for s,sample in enumerate(samples):
    print(f'1000*area for {freq_strs[s]}: {1000*ellipse_area(sample)}')


### HILC (weights vary) Ellipse Areas

In [ ]:
freq_strs = ['90_120', '90_150', '90_280', '90_120_150', '90_150_280', '90_150_220', '280_353', '280_353_400']

samples = []
for i, freq_str in enumerate(freq_strs):
    samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_HILC_{tsz_type}tsz_weightsvary_{Nsims//1000}ksims_tszamp150_gaussianlkl_SR.p', 'rb')).T, names = names, labels = labels, label=f'HILC (weights vary) {freq_str.replace("_", ", ")}'))

for s,sample in enumerate(samples):
    print(f'1000*area for {freq_strs[s]}: {1000*ellipse_area(sample)}')


### NILC Ellipse Areas

In [ ]:
freq_strs = ['90_120', '90_150', '90_280', '90_120_150', '90_150_280', '90_150_220', '280_353', '280_353_400']

samples = []
for i, freq_str in enumerate(freq_strs):
    samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_NILC_{tsz_type}tsz_{Nsims//1000}ksims_tszamp150_gaussianlkl_4scales.p', 'rb')).T, names = names, labels = labels, label=f'NILC {freq_str.replace("_", ", ")}'))

for s,sample in enumerate(samples):
    print(f'1000*area for {freq_strs[s]}: {1000*ellipse_area(sample)}')


# Full Comparison LFI

## Posteriors

In [ ]:
Nsims = 30000
Nsims_backup = 40000

In [ ]:
freq_strs = ['90_150', '90_120', '90_120_150', '90_280', '90_150_220', '90_150_280', '280_353', '280_353_400']


for freq_str in freq_strs:
    samples = []
    try:
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_multifrequency_{tsz_type}tsz_{Nsims//1000}ksims_tszamp150_lfi.p', 'rb')).T, names = names, labels = labels, label=f'Mult. PS LFI {freq_str.replace("_", ", ")}'))
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_HILC_{tsz_type}tsz_weightsonce_{Nsims//1000}ksims_tszamp150_lfi.p', 'rb')).T, names = names, labels = labels, label=f'HILC (weights once) LFI {freq_str.replace("_", ", ")}'))
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_HILC_{tsz_type}tsz_weightsvary_{Nsims//1000}ksims_tszamp150_lfi.p', 'rb')).T, names = names, labels = labels, label=f'HILC (weights vary) LFI {freq_str.replace("_", ", ")}'))
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_NILC_{tsz_type}tsz_{Nsims//1000}ksims_tszamp150_lfi_4scales.p', 'rb')).T, names = names, labels = labels, label=f'NILC LFI {freq_str.replace("_", ", ")}'))
    except FileNotFoundError:
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_multifrequency_{tsz_type}tsz_{Nsims_backup//1000}ksims_tszamp150_lfi.p', 'rb')).T, names = names, labels = labels, label=f'Mult. PS LFI {freq_str.replace("_", ", ")}'))
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_HILC_{tsz_type}tsz_weightsonce_{Nsims_backup//1000}ksims_tszamp150_lfi.p', 'rb')).T, names = names, labels = labels, label=f'HILC (weights once) LFI {freq_str.replace("_", ", ")}'))
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_HILC_{tsz_type}tsz_weightsvary_{Nsims_backup//1000}ksims_tszamp150_lfi.p', 'rb')).T, names = names, labels = labels, label=f'HILC (weights vary) LFI {freq_str.replace("_", ", ")}'))
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_NILC_{tsz_type}tsz_{Nsims_backup//1000}ksims_tszamp150_lfi_4scales.p', 'rb')).T, names = names, labels = labels, label=f'NILC LFI {freq_str.replace("_", ", ")}'))
    
    
    g = plots.get_subplot_plotter()
    g.triangle_plot(samples, filled=True)
    plt.savefig(f'{freq_str}.pdf')

### Multifrequency

In [ ]:
freq_strs = ['90_150', '90_120_150', '90_280', '90_150_220', '90_150_280', '280_353', '280_353_400']

samples = []
for freq_str in freq_strs:
    try:
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_multifrequency_{tsz_type}tsz_{Nsims//1000}ksims_tszamp150_lfi.p', 'rb')).T, names = names, labels = labels, label=f'Mult. PS LFI {freq_str}'))
    except FileNotFoundError:
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_multifrequency_{tsz_type}tsz_{Nsims_backup//1000}ksims_tszamp150_lfi.p', 'rb')).T, names = names, labels = labels, label=f'Mult. PS LFI {freq_str}'))   
    
g = plots.get_subplot_plotter()
g.triangle_plot(samples, filled=True)
plt.savefig('mult_lfi_freq_comparison.pdf')


### HILC (weights once)

In [ ]:
freq_strs = ['90_150', '90_120_150', '90_280', '90_150_220', '90_150_280', '280_353', '280_353_400']

samples = []
for freq_str in freq_strs:
    try:
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_HILC_{tsz_type}tsz_weightsonce_{Nsims//1000}ksims_tszamp150_lfi.p', 'rb')).T, names = names, labels = labels, label=f'HILC (weights once) LFI {freq_str}'))
    except FileNotFoundError:
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_HILC_{tsz_type}tsz_weightsonce_{Nsims_backup//1000}ksims_tszamp150_lfi.p', 'rb')).T, names = names, labels = labels, label=f'HILC (weights once) LFI {freq_str}'))
    
g = plots.get_subplot_plotter()
g.triangle_plot(samples, filled=True)
plt.savefig('hilc_wtsonce_lfi_freq_comparison.pdf')

### HILC (weights vary)

In [ ]:
freq_strs = ['90_150', '90_120_150', '90_280', '90_150_220', '90_150_280', '280_353', '280_353_400']

samples = []
for freq_str in freq_strs:
    try:
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_HILC_{tsz_type}tsz_weightsvary_{Nsims//1000}ksims_tszamp150_lfi.p', 'rb')).T, names = names, labels = labels, label=f'HILC (weights vary) LFI {freq_str}'))
    except FileNotFoundError:
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_HILC_{tsz_type}tsz_weightsvary_{Nsims_backup//1000}ksims_tszamp150_lfi.p', 'rb')).T, names = names, labels = labels, label=f'HILC (weights vary) LFI {freq_str}'))  
    
g = plots.get_subplot_plotter()
g.triangle_plot(samples, filled=True)
plt.savefig('hilc_wtsvary_lfi_freq_comparison.pdf')

### NILC

In [ ]:
freq_strs = ['90_150', '90_120_150', '90_280', '280_353', '280_353_400' , '90_150_280', '90_150_220']
samples = []
for freq_str in freq_strs:
    try:
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_NILC_{tsz_type}tsz_{Nsims//1000}ksims_tszamp150_lfi_4scales.p', 'rb')).T, names = names, labels = labels, label=f'NILC LFI {freq_str.replace("_", ", ")}'))
    except FileNotFoundError:
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_NILC_{tsz_type}tsz_{Nsims_backup//1000}ksims_tszamp150_lfi_4scales.p', 'rb')).T, names = names, labels = labels, label=f'NILC LFI {freq_str.replace("_", ", ")}'))  
    
g = plots.get_subplot_plotter()
g.triangle_plot(samples, filled=True)
plt.savefig('nilc_lfi_freq_comparison.pdf')

## Ellipse Areas

### Multifrequency Ellipse Areas

In [ ]:
freq_strs = ['90_120', '90_150', '90_280', '90_120_150', '90_150_280', '90_150_220', '280_353', '280_353_400']
samples = []
for freq_str in freq_strs:
    try:
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_multifrequency_{tsz_type}tsz_{Nsims//1000}ksims_tszamp150_lfi.p', 'rb')).T, names = names, labels = labels, label=f'Mult. PS LFI {freq_str}'))
    except FileNotFoundError:
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_multifrequency_{tsz_type}tsz_{Nsims_backup//1000}ksims_tszamp150_lfi.p', 'rb')).T, names = names, labels = labels, label=f'Mult. PS LFI {freq_str}'))

for s,sample in enumerate(samples):
    print(f'1000*area for {freq_strs[s]}: {1000*ellipse_area(sample)}')

### HILC (weights once) Ellipse Areas

In [ ]:
freq_strs = ['90_120', '90_150', '90_280', '90_120_150', '90_150_280', '90_150_220', '280_353', '280_353_400']
samples = []
for freq_str in freq_strs:
    try:
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_HILC_{tsz_type}tsz_weightsonce_{Nsims//1000}ksims_tszamp150_lfi.p', 'rb')).T, names = names, labels = labels, label=f'HILC (weights once) LFI {freq_str}'))
    except FileNotFoundError:
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_HILC_{tsz_type}tsz_weightsonce_{Nsims_backup//1000}ksims_tszamp150_lfi.p', 'rb')).T, names = names, labels = labels, label=f'HILC (weights once) LFI {freq_str}'))

for s,sample in enumerate(samples):
    print(f'1000*area for {freq_strs[s]}: {1000*ellipse_area(sample)}')

### HILC (weights vary) Ellipse Areas

In [ ]:
freq_strs = ['90_120', '90_150', '90_280', '90_120_150', '90_150_280', '90_150_220', '280_353', '280_353_400']
samples = []
for freq_str in freq_strs:
    try:
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_HILC_{tsz_type}tsz_weightsvary_{Nsims//1000}ksims_tszamp150_lfi.p', 'rb')).T, names = names, labels = labels, label=f'HILC (weights vary) LFI {freq_str}'))
    except FileNotFoundError:
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_HILC_{tsz_type}tsz_weightsvary_{Nsims_backup//1000}ksims_tszamp150_lfi.p', 'rb')).T, names = names, labels = labels, label=f'HILC (weights vary) LFI {freq_str}'))

for s,sample in enumerate(samples):
    print(f'1000*area for {freq_strs[s]}: {1000*ellipse_area(sample)}')

### NILC Ellipse Areas

In [ ]:
freq_strs = ['90_120', '90_150', '90_280', '90_120_150', '90_150_280', '90_150_220', '280_353', '280_353_400']
samples = []
for freq_str in freq_strs:
    try:
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_NILC_{tsz_type}tsz_{Nsims//1000}ksims_tszamp150_lfi_4scales.p', 'rb')).T, names = names, labels = labels, label=f'NILC LFI {freq_str}'))
    except FileNotFoundError:
        samples.append(MCSamples(samples=pickle.load(open(f'/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_{freq_str}/posteriors/a_array_NILC_{tsz_type}tsz_{Nsims_backup//1000}ksims_tszamp150_lfi_4scales.p', 'rb')).T, names = names, labels = labels, label=f'NILC LFI {freq_str}'))

for s,sample in enumerate(samples):
    print(f'1000*area for {freq_strs[s]}: {1000*ellipse_area(sample)}')